# Lab 12-4 many to many variable
### simple pos-tagger training 
* many to many
* variable input sequence length

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
tf.set_random_seed(777)

### Prepairing dataset

In [2]:
sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
pos = [['pronoun', 'verb', 'adjective'],
     ['noun', 'verb', 'adverb', 'adjective'],
     ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
     ['noun', 'verb', 'adverb', 'adjective', 'verb']]

### Preprocessing dataset

In [3]:
# word의 dictionary
word_list = sum(sentences, [])
word_list.sort()
word_list = ['<pad>'] + word_list
word2idx = {word : idx for idx, word in enumerate(word_list)}
idx2word = {idx : word for idx, word in enumerate(word_list)}

print(word2idx)
print(idx2word)

{'<pad>': 0, 'I': 1, 'a': 2, 'changing': 3, 'deep': 4, 'difficult': 5, 'fast': 6, 'feel': 7, 'for': 8, 'framework': 9, 'hungry': 10, 'is': 13, 'learning': 14, 'tensorflow': 17, 'very': 19}
{0: '<pad>', 1: 'I', 2: 'a', 3: 'changing', 4: 'deep', 5: 'difficult', 6: 'fast', 7: 'feel', 8: 'for', 9: 'framework', 10: 'hungry', 11: 'is', 12: 'is', 13: 'is', 14: 'learning', 15: 'tensorflow', 16: 'tensorflow', 17: 'tensorflow', 18: 'very', 19: 'very'}


In [4]:
# pos (part of speech)의 dictionary
pos_list = sum(pos, [])
pos_list.sort()
pos_list = ['<pad>'] + pos_list
pos2idx = {pos : idx for idx, pos in enumerate(pos_list)}
idx2pos = {idx : pos for idx, pos in enumerate(pos_list)}

print(pos2idx)
print(idx2pos)

{'<pad>': 0, 'adjective': 4, 'adverb': 6, 'determiner': 7, 'noun': 12, 'preposition': 13, 'pronoun': 14, 'verb': 19}
{0: '<pad>', 1: 'adjective', 2: 'adjective', 3: 'adjective', 4: 'adjective', 5: 'adverb', 6: 'adverb', 7: 'determiner', 8: 'noun', 9: 'noun', 10: 'noun', 11: 'noun', 12: 'noun', 13: 'preposition', 14: 'pronoun', 15: 'verb', 16: 'verb', 17: 'verb', 18: 'verb', 19: 'verb'}


In [5]:
max_length = 10
x_data = list(map(lambda sentence : [word2idx.get(token) for token in sentence], sentences))
x_data = pad_sequences(sequences = x_data, maxlen = max_length, padding = 'post', truncating = 'post')
x_len = list(map(lambda sentence : len(sentence), sentences))

y_data = list(map(lambda sentence : [pos2idx.get(token) for token in sentence], pos))
y_data = pad_sequences(sequences = y_data, maxlen = max_length, padding = 'post', truncating = 'post')

### Creating graph

In [6]:
# hyper-parameters
epochs = 10
batch_size = 2
lr = 0.5

# input
data = tf.data.Dataset.from_tensor_slices((x_len, x_data, y_data))
data = data.shuffle(buffer_size = 10)
data = data.batch(batch_size = batch_size)
iterator = data.make_initializable_iterator()
x_mb_len, x_mb, y_mb = iterator.get_next()

In [7]:
# hyper-parameters for lstm (many to many), one-hot encoding
n_of_classes = len(idx2pos)
hidden_size = 10

one_hot_encoding = tf.eye(num_rows = len(idx2word), dtype = tf.float32)
one_hot_encoding = tf.get_variable(name = 'one_hot_encoding',
                                   initializer = one_hot_encoding, trainable = False)

x_mb_batch = tf.nn.embedding_lookup(params = one_hot_encoding, ids = x_mb)
lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units = hidden_size, dtype = tf.float32)
score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell = lstm_cell, output_size = n_of_classes)
outputs, _ = tf.nn.dynamic_rnn(cell = score_cell, inputs = x_mb_batch, dtype = tf.float32,
                               sequence_length = x_mb_len)

In [8]:
# loss
masking = tf.sequence_mask(lengths = x_mb_len, maxlen = max_length, dtype = tf.float32)
s2s_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = y_mb, weights = masking)
prediction = tf.argmax(input = outputs, axis = -1)

In [9]:
# training
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = s2s_loss)

### Training

In [10]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

In [11]:
tr_loss_hist = []

for epoch in range(epochs):
    
    sess.run(iterator.initializer)
    avg_tr_loss = 0
    step = 0
    
    try:
        while True:
            _, loss = sess.run([training_op, s2s_loss])
            avg_tr_loss += loss
            step += 1
    except:
        avg_tr_loss /= step
        tr_loss_hist.append(avg_tr_loss)
        
    print('epoch : {:3}, loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, loss : 2.450
epoch :   2, loss : 2.020
epoch :   3, loss : 1.266
epoch :   4, loss : 0.607
epoch :   5, loss : 0.361
epoch :   6, loss : 0.217
epoch :   7, loss : 0.134
epoch :   8, loss : 0.089
epoch :   9, loss : 0.052
epoch :  10, loss : 0.027


### Accuracy

In [12]:
yhat = sess.run(prediction, feed_dict = {x_mb_len : x_len, x_mb : x_data})

In [13]:
print('prediction : {}'.format(yhat))

prediction : [[14 19  4  0  0  0  0  0  0  0]
 [12 19  6  4  0  0  0  0  0  0]
 [12 19  7 12 13  4 12  0  0  0]
 [12 19  6  4 19  0  0  0  0  0]]


In [14]:
print('true : {}'.format(y_data))

true : [[14 19  4  0  0  0  0  0  0  0]
 [12 19  6  4  0  0  0  0  0  0]
 [12 19  7 12 13  4 12  0  0  0]
 [12 19  6  4 19  0  0  0  0  0]]


In [15]:
print('accuracy : {:.2%}'.format(np.mean(np.all(yhat == y_data, axis = -1))))

accuracy : 100.00%
